# 강좌 엑셀 파일에서 불러오기

In [15]:
# from urllib.request import urlopen
# html = urlopen("https://www.frip.co.kr/daily/learning/all?orderBy=popular&page=1")  
# bsObject = BeautifulSoup(html, "html.parser")
# print(bsObject)

In [39]:
import requests
from bs4 import BeautifulSoup

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
url = "https://www.frip.co.kr/products/108923"
webpage = requests.get(url, headers=headers)
soup = BeautifulSoup(webpage.content, "html.parser")
soup.select(
'body')
# print(soup)

[<body><noscript>If you're seeing this message, that means <strong>JavaScript has been disabled on your browser</strong>, please <strong>enable JS</strong> to make this app work.</noscript><div id="app"></div><div id="modal-root"></div><div id="dnd-root"></div><link href="https://fonts.googleapis.com/css?family=Noto+Sans+KR:300,400,500,700" rel="stylesheet"/><script src="/runtime~main.bcf9f3a8eff7fa7c2423.js" type="text/javascript"></script><script src="/vendors~main.3749da4eac8b61e1a652.chunk.js" type="text/javascript"></script><script src="/main.680bb431ba4e89ad202b.chunk.js" type="text/javascript"></script></body>]

In [8]:
with open('lecture.csv') as file:
    csv_data = []
    for line in file.readlines():
        csv_data.append(line.split(','))
print(csv_data[0][0])

강좌명


In [ ]:
import numpy
csv_data = numpy.loadtxt('lecture.csv', delimiter=',')


In [ ]:
import pandas as pd
csv_data = pd.read_csv("./lecture.csv")
print(csv_data)

# 셀리니움을 활용하여 frip에서 크롤링

In [42]:
pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge selenium

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\sehyu\anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    selenium-3.141.0           |py37h8055547_1001         937 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m
  selenium           conda-forge/win-64::selenium-3.141.0-py37h8055547_1001

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.2-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1




p



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




In [1]:
import time
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime

driver = webdriver.Chrome('./chromedriver')

## 페이지 넘어가면서 크롤링

In [2]:
links = []
for i in range(1, 179):
    driver.get('https://www.frip.co.kr/daily/learning/all?page='+ str(i))
    time.sleep(1)
    lectures = driver.find_elements_by_class_name("sc-kGXeez")
    for lecture in lectures:
        a_name = lecture.find_element_by_tag_name('a').get_attribute('href')
        links.append(a_name)

# 크롤링한 페이지 url csv파일로 저장

## DataFrame 방식으로 저장

In [3]:
import pandas as pd

dataframe = pd.DataFrame(links)
dataframe.to_csv('./frip.csv', header=False,index=False)

## csv writer 방식으로 저장

In [4]:
import csv

In [150]:
f = open('./frip_csv.csv', 'w', newline='')
wr = csv.writer(f)
for link in links:
    wr.writerow([link])
f.close()

# 페이지 url이 저장된 csv 파일 불러오기

In [5]:
f = open('./frip.csv', 'r')
rdr = csv.reader(f)
new_links = []
for line in rdr:
    new_links.append(line[0])
f.close()

In [151]:
f = open('./frip_csv.csv', 'r')
rdr = csv.reader(f)
new_links = []
for line in rdr:
    new_links.append(line[0])
f.close()

In [6]:
len(new_links)

3554

# 페이지 별로 돌아가면서 데이터 크롤링

## 한개만 테스트

In [12]:
driver.get("https://www.frip.co.kr/products/134289")
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
meeting_id = "https://www.frip.co.kr/products/114211".split('/')[-1]
uid = driver.find_element_by_class_name('nCgAM').find_element_by_tag_name('a').get_attribute('href')
writer = soup.select_one('span.gYdgbL')
created_at = datetime.now()
updated_at = datetime.now()
is_period = 0
meeting_date = datetime.now()
temp = soup.select('section.sc-dUcZlc > div.sc-ZUflv > div.sc-esoVGF')
if temp[1].text[0].isdigit():
    temp_date = ''
    for t in temp[1].text:
        if t == '외':
            break
        if t in '(월화수목금토일요일년)':
            continue
        if t == ' ':
            t = '-'
        temp_date += t
    temp_date = temp_date[:-1]
    meeting_date = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
period_date = ''
is_class = 1
max_person = 20
now_person = 1
content = '[내용]' + soup.select('div.cSNowP')[0].text
ref_url = new_links[0]
address = soup.select_one('div.sc-kRCAcj').text[:-5]
fee = soup.select_one('div.sc-eBipZS').text[:-1].replace(',', '')
unit = '원'
is_active = 1
like_temp = ''
for t in temp[0].text:
    if not t.isdigit():
        break
    like_temp += t
like_cnt = int(like_temp)
view_cnt = 0
score = 0
if temp[3].text[6].isdigit():
    score = float(temp[3].text[6:])
main_category = 0
tags = ''

IndexError: string index out of range

In [175]:
meetings.append(
    [
        meeting_id,
        uid,
        writer,
        created_at,
        updated_at,
        is_period,
        meeting_date,
        period_date,
        is_class,
        max_person,
        now_person,
        content,
        ref_url,
        address,
        fee,
        unit,
        is_active,
        like_cnt,
        view_cnt,
        score,
        main_category,
        tags
    ]
)

## 미팅 시간 연산

In [85]:
from datetime import datetime

temp_date = ''
for t in temp[1].text:
    if t == '외':
        break
    if t in '(월화수목금토일요일년)':
        continue
    if t == ' ':
        t = '-'
    temp_date += t
temp_date = temp_date[:-1]
datetime_object = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
print(datetime_object)

2020-05-23 13:00:00


## columns 생성

In [2]:
import pandas as pd
from datetime import datetime

meeting_columns = (
    "meeting_id",
    "uid",
    "writer",
    "created_at",
    "updated_at",
    "is_period",
    "meeting_date",
    "period_date",
    "is_class",
    "max_person",
    "now_person",
    "content",
    "ref_url",
    "address",
    "fee",
    "unit",
    "is_active",
    "like_cnt",
    "view_cnt",
    "score",
    "main_category",
    "tags",
    "title",
    "area1",
    "area2"
)

image_columns = (
    "meeting_id",
    "image_url"
)

## 제대로 테스트

In [38]:
meetings = []
images = []

In [41]:
for link in range(3081, len(new_links)):
    print(new_links[link])
    driver.get(new_links[link])
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    meeting_id = new_links[link].split('/')[-1]
    uid = driver.find_element_by_class_name('nCgAM').find_element_by_tag_name('a').get_attribute('href')
    writer = soup.select_one('span.gYdgbL').text
    created_at = datetime.now()
    updated_at = datetime.now()
    is_period = 0
    meeting_date = datetime.now()
    temp = soup.select('section.sc-dUcZlc > div.sc-ZUflv > div.sc-esoVGF')
    if temp[-3].text == '':
        continue
    elif temp[-3].text[0].isdigit():
        temp_date = ''
        for t in temp[-3].text:
            if t == '외':
                break
            if t in '(월화수목금토일요일년)':
                continue
            if t == ' ':
                t = '-'
            temp_date += t
        temp_date = temp_date[:-1]
        try:
            meeting_date = datetime.strptime(temp_date, '%Y-%m-%d-%H:%M')
        except:
            meeting_date = datetime.now()
    period_date = ''
    is_class = 1
    max_person = 20
    now_person = 1
    content = '[내용]' + soup.select('div.cSNowP')[0].text
    ref_url = new_links[link]
    print(soup.select_one('div.sc-kRCAcj').text[:-5])
    address = soup.select_one('div.sc-kRCAcj').text[:-5]
    fee = soup.select_one('div.sc-eBipZS').text[:-1].replace(',', '')
    unit = '원'
    is_active = 1
    like_temp = ''
    for t in temp[0].text:
        if not t.isdigit():
            break
        like_temp += t
    try:
        like_cnt = int(like_temp)
    except:
        like_cnt = 0
    view_cnt = 0
    score = 0
    if temp[-1].text[6].isdigit():
        score = float(temp[-1].text[6:])
    main_category = 0
    tags = ''
    title = soup.select_one('h1.jVCVBk').text
    if len(soup.select_one('div.sc-kRCAcj').text[:-5].split(' ')) > 1:
        area1 = soup.select_one('div.sc-kRCAcj').text[:-5].split(' ')[0]
        area2 = soup.select_one('div.sc-kRCAcj').text[:-5].split(' ')[1]
    else:
        area1 = ''
        area2 = ''
    
    # 이미지 저장
    images_link = driver.find_elements_by_class_name('cljRKC')
    image_urls = set()
    for i in images_link:
        image_urls.add(str(i.get_attribute('src')))
    image_url = ' '.join(image_urls)
    
    # 데이터 저장
    meetings.append(
    [
        meeting_id,
        uid,
        writer,
        created_at,
        updated_at,
        is_period,
        meeting_date,
        period_date,
        is_class,
        max_person,
        now_person,
        content,
        ref_url,
        address,
        fee,
        unit,
        is_active,
        like_cnt,
        view_cnt,
        score,
        main_category,
        tags,
        title,
        area1,
        area2
        
    ]
    )
        
    images.append(
    [
        meeting_id,
        image_url
    ]
    )


https://www.frip.co.kr/products/119443
서울시 강북구 한천로129길 5, 지하 1층
https://www.frip.co.kr/products/119435
서울시 송파구 위례광장로 170 상가 A동 209호
https://www.frip.co.kr/products/119395
경기도 성남시 분당구 정자동 48-10, D.island EnglishCafe
https://www.frip.co.kr/products/119251
서울시 마포구 마포대로21길38, 1층
https://www.frip.co.kr/products/111983
서울 송파구 올림픽로 424 올림픽 수영장(올림픽공원 내 위치함)
https://www.frip.co.kr/products/118470
서울시 마포구 마포대로21길38, 1층
https://www.frip.co.kr/products/116864
서울시 강남구 선릉로68길 27 102호 벨로르플라워
https://www.frip.co.kr/products/118093
서울시 영등포구 문래동2가 14-91 일럼
https://www.frip.co.kr/products/117380
서울시 성동구 성수동 1가 656-834 카페 라이트하우스
https://www.frip.co.kr/products/117139
서울시 강남구 역삼동 777-40 (지하 1층)
https://www.frip.co.kr/products/108698
서울 관악구 봉천동 1666-68 지하 1층 하루아키아트공방
https://www.frip.co.kr/products/109264
서울시 (협의 후 별도 공지 예정)
https://www.frip.co.kr/products/108944
서울시 관악구 신림동 1433-170 지하1층 작업실
https://www.frip.co.kr/products/111446
서울시 서대문구 연희동 446-285, 4층
https://www.frip.co.kr/products/116630
성남시 수정구 위례순환로

서울시 동작구 동작대로 49 셀스 스터디룸
https://www.frip.co.kr/products/134855
온라인
https://www.frip.co.kr/products/135006
서울시 종로구 사직로 64-1 남양빌딩 3층, 301호
https://www.frip.co.kr/products/135004
서울시 종로구 사직로 64-1 남양빌딩 3층, 301호
https://www.frip.co.kr/products/134980
배송상품
https://www.frip.co.kr/products/134942
온라인
https://www.frip.co.kr/products/134747
온라인
https://www.frip.co.kr/products/134598
서울시 송파구 위례광장로 170, 상가 a동 209호
https://www.frip.co.kr/products/134899
서울 마포구 모래내로3길 11 상암미르웰 아담한 홈공방
https://www.frip.co.kr/products/134831
서울 마포구 합정동 431-42 B1 실버라스튜디오
https://www.frip.co.kr/products/130143
온라인
https://www.frip.co.kr/products/134748
서울특별시 마포구 합정동 431-42, B1 실버라스튜디오
https://www.frip.co.kr/products/134565
대전(근린공원),  서울(용마폭포/올림픽/용산가족공원)
https://www.frip.co.kr/products/134627
경기도 성남시 분당구 황새울로116번길 16, 1층 버블다움
https://www.frip.co.kr/products/133306
온라인
https://www.frip.co.kr/products/134412
서울시 강남구 테헤란로8길 28 2층 모아트 스튜디오
https://www.frip.co.kr/products/133730
서울시 강남구 테헤란로8길 28 2층 모아트 스튜디오
https://www.frip.

서울시 송파구 위례광장로 170, 힐스테이트몰 A동 209호
https://www.frip.co.kr/products/127338
경기도 의정부시 의정부동 135-25, 2층
https://www.frip.co.kr/products/126748
서울 성북구 성북로23나길 4, 아임그린게스트하우스
https://www.frip.co.kr/products/127131
서울시 강서구 방화대로48길 40, 217동
https://www.frip.co.kr/products/127120
서울시 강남구 언주로30길 57, 타워팰리스 E동 연회장
https://www.frip.co.kr/products/124536
서울시 종로구 계동길 121, 1,3,4층
https://www.frip.co.kr/products/126478
강남역/사당역/당산역/종각역/신촌역/이대역/홍대역/혜화역/건대역/잠실새내역/압구정역/부산
https://www.frip.co.kr/products/127059
서울 용산구 회나무로39길 6-17 , 1층 홍실
https://www.frip.co.kr/products/127101
서울시 광진구 천호대로121길 53, 1F 2좌
https://www.frip.co.kr/products/127074
서울시 마포구 성미산로28길 31 1층, 소피네잡화점
https://www.frip.co.kr/products/126916
대구 달성군 유가읍 테크노대로 195 (인근 카페로 협의 후 진행)
https://www.frip.co.kr/products/126155
서울시 서초구 강남대로 459, 3층
https://www.frip.co.kr/products/126907
서울시 용산구 회나무로13길 12, 4층 꼬띠디앙
https://www.frip.co.kr/products/126682
서울시 강남구 논현동 172-22, 1층
https://www.frip.co.kr/products/126798
서울시 광진구 능동로 276 8층
https://www.frip.co.k

서울시 영등포구 도림로112길 35 (1층 및 지하 1층)
https://www.frip.co.kr/products/117349
서울 마포구 연남동 249-7 오후의 작업실
https://www.frip.co.kr/products/110079
서울시 서대문구 북가좌동 377-34
https://www.frip.co.kr/products/117347
서울시 관악구 봉천로 522 3층
https://www.frip.co.kr/products/117314
경기도 남양주시 화도읍 경춘로2290번길3 월산부영아파트2단지상가1동 201호 (CU편의점2층)
https://www.frip.co.kr/products/117343
경기 화성시 동탄치동천로3길 3-13 102호
https://www.frip.co.kr/products/117218
온라인(슬랙 메신저 사용)
https://www.frip.co.kr/products/116855
서울 서초구 서초동 1483-2
https://www.frip.co.kr/products/111250
서울시 송파구 문정동 50-12 댄싱위드플라워
https://www.frip.co.kr/products/116868
서울시 송파구 송파대로 458 해광빌딩 204호 아트블라썸
https://www.frip.co.kr/products/116247
서울시 송파구 송파동 54-1, 1층
https://www.frip.co.kr/products/110827
서울시 송파구 잠실동 221-13 204호
https://www.frip.co.kr/products/109003
서울시 서초구 사평대로52길 13 지하 스튜디오 37
https://www.frip.co.kr/products/109019
서울시 용산구 한남동 683-611층 아이리에 가죽공방
https://www.frip.co.kr/products/108870
서울시 영등포구 문래동2가 14-91
https://www.frip.co.kr/products/116054
서울시 마포구 연남동 240-16

AttributeError: 'NoneType' object has no attribute 'text'

In [42]:
meeting_frame = pd.DataFrame(data=meetings, columns=meeting_columns)
meeting_frame

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,unit,is_active,like_cnt,view_cnt,score,main_category,tags,title,area1,area2
0,108923,https://www.frip.co.kr/hosts/617,그 아저씨,2020-05-30 20:58:48.363822,2020-05-30 20:58:48.363822,0,2020-05-30 20:58:48.363822,,1,20,...,원,1,11222,0,4.77,0,,[널위한선물] 도예 공방 원데이 / 도자기 그릇 만들기 (예약가능),서울시,성동구
1,133413,https://www.frip.co.kr/hosts/5545,MCJ,2020-05-30 20:58:52.561736,2020-05-30 20:58:52.561736,0,2020-06-03 19:30:00.000000,,1,20,...,원,1,265,0,5.00,0,,BYOB 각자 와인 1병씩 가져오는 MCJ 홈파티,서울시,강남구
2,101774,https://www.frip.co.kr/hosts/746,아이홉 맥주공방 (I'HOP),2020-05-30 20:58:55.192702,2020-05-30 20:58:55.192702,0,2020-05-31 13:00:00.000000,,1,20,...,원,1,9848,0,4.76,0,,[가정의달] 술술 놀면서 수제 맥주 만들기,서울시,송파구
3,133023,https://www.frip.co.kr/hosts/10654,비푸머스,2020-05-30 20:58:58.940685,2020-05-30 20:58:58.940685,0,2020-05-30 20:58:58.940685,,1,20,...,원,1,888,0,4.81,0,,[서울/홍대] 최상급 프랑스 향료로 나만의 향수 만들기 (예약 가능),서울시,마포구
4,115497,https://www.frip.co.kr/hosts/2839,지아네요리공작소,2020-05-30 20:59:02.833314,2020-05-30 20:59:02.833314,0,2020-05-30 20:59:02.833314,,1,20,...,원,1,5674,0,4.89,0,,[망원동] 스콘 2종 원데이클래스 (예약 가능),서울시,마포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3545,115820,https://www.frip.co.kr/hosts/579,블루기타,2020-05-31 02:48:36.206963,2020-05-31 02:48:36.206963,0,2020-05-31 02:48:36.206963,,1,20,...,원,1,266,0,0.00,0,,[4주] 한 달 만에 드럼연주 마스터하기 (예약 가능),서울시,영등포구
3546,116002,https://www.frip.co.kr/hosts/862,가든오브캔들,2020-05-31 02:48:38.774150,2020-05-31 02:48:38.774150,0,2020-05-31 02:48:38.774150,,1,20,...,원,1,60,0,0.00,0,,친환경 재료로 만드는 화분 도우아트(예약 가능),서울시,관악구
3547,115973,https://www.frip.co.kr/hosts/162,꽃보다바둑센터,2020-05-31 02:48:41.673348,2020-05-31 02:48:41.673348,0,2020-05-31 02:48:41.673348,,1,20,...,원,1,26,0,0.00,0,,지도대국 1회권 (예약 가능),꽃보다,바둑센터
3548,115982,https://www.frip.co.kr/hosts/711,FuNOD,2020-05-31 02:48:44.268411,2020-05-31 02:48:44.268411,0,2020-05-31 02:48:44.268411,,1,20,...,원,1,186,0,0.00,0,,[4주] 영문 캘리그라피 클래스 (예약 가능),서울시,용산구


In [43]:
meeting_frame.to_csv('./meeting_data.csv', header=True,index=False, encoding='utf-8-sig')

In [36]:
meeting_frame.tail(100)

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,unit,is_active,like_cnt,view_cnt,score,main_category,tags,title,area1,area2
716,124335,https://www.frip.co.kr/hosts/8394,예소피아노,2020-05-30 19:27:55.160937,2020-05-30 19:27:55.160937,0,2020-05-30 19:27:55.160937,,1,20,...,원,1,321,0,4.96,0,,[NEW최신가요출시] 똥꼬발랄 예소쌤과 대환장 피아노원데이클래스 1일1곡 완성시키기...,서울시,영등포구
717,108410,https://www.frip.co.kr/hosts/5324,김아이캘리그라피,2020-05-30 19:27:59.582936,2020-05-30 19:27:59.582936,0,2020-05-30 19:27:59.582936,,1,20,...,원,1,997,0,4.90,0,,[원데이] 손글씨로 나만의 캘리그라피 액자 만들기 (예약 가능),홍대입구역,근처
718,112131,https://www.frip.co.kr/hosts/500,유어 하루드로잉,2020-05-30 19:28:02.391937,2020-05-30 19:28:02.391937,0,2020-06-02 19:30:00.000000,,1,20,...,원,1,5400,0,4.89,0,,여행지 펜 드로잉,서울시,마포구
719,115615,https://www.frip.co.kr/hosts/1609,앙센베이킹,2020-05-30 19:28:07.051248,2020-05-30 19:28:07.051248,0,2020-05-30 19:28:07.051248,,1,20,...,원,1,219,0,4.86,0,,레몬 케이크 만들기 (예약 가능),서울시,중랑구
720,118019,https://www.frip.co.kr/hosts/519,레더레시피🌿,2020-05-30 19:28:11.400245,2020-05-30 19:28:11.400245,0,2020-05-30 19:28:11.400245,,1,20,...,원,1,197,0,4.83,0,,직접 만드는 가죽 여권케이스 (예약 가능),서울시,용산구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,113810,https://www.frip.co.kr/hosts/6528,PPT마스터,2020-05-30 19:34:25.954343,2020-05-30 19:34:25.954343,0,2020-06-06 11:00:00.000000,,1,20,...,원,1,1216,0,4.94,0,,상위 1%의 광고인이 알려주는 PPT신공!,강남역,인근
812,120929,https://www.frip.co.kr/hosts/7621,봉봉주스,2020-05-30 19:34:29.694343,2020-05-30 19:34:29.694343,0,2020-05-30 19:34:29.694343,,1,20,...,원,1,257,0,4.91,0,,"커피의 모든 것, 추출부터 라떼아트까지 (예약 가능)",서울,영등포구
813,109373,https://www.frip.co.kr/hosts/3456,샤당구클럽,2020-05-30 19:34:33.432348,2020-05-30 19:34:33.432348,0,2020-05-30 19:34:33.432348,,1,20,...,원,1,547,0,4.81,0,,1:1 포켓볼 집중 강습! (예약 가능),서울시,관악구
814,115452,https://www.frip.co.kr/hosts/242,에코다이버스,2020-05-30 19:34:38.200345,2020-05-30 19:34:38.200345,0,2020-05-30 19:34:38.200345,,1,20,...,원,1,140,0,0.50,0,,[제주] 신비로운 제주 바다! 1박2일 다이빙 자격증 완파하기!,제주도,서귀포시


# 이미지 데이터 저장

In [45]:
image_frame = pd.DataFrame(data=images, columns=image_columns)

In [38]:
image_frame = pd.read_csv('./meeting_images.csv')

In [46]:
image_frame.to_csv('./meeting_images.csv', header=True, encoding='utf-8-sig')

In [10]:
meeting_frame = pd.read_csv('./meeting_data.csv')

In [47]:
meeting_frame.dtypes

meeting_id               object
uid                      object
writer                   object
created_at       datetime64[ns]
updated_at       datetime64[ns]
is_period                 int64
meeting_date     datetime64[ns]
period_date              object
is_class                  int64
max_person                int64
now_person                int64
content                  object
ref_url                  object
address                  object
fee                      object
unit                     object
is_active                 int64
like_cnt                  int64
view_cnt                  int64
score                   float64
main_category             int64
tags                     object
title                    object
area1                    object
area2                    object
dtype: object

# writer 이름 이상했던거 수정(이제 필요 없음)

In [6]:
writers = []
for i, m in meeting_frame.iterrows():
    tmp = m.writer[31:]
    tmp = tmp[:-7]
    writers.append(tmp)
writers

['그 아저씨',
 'MCJ',
 "아이홉 맥주공방 (I'HOP)",
 '비푸머스',
 '지아네요리공작소',
 '포임스포츠클럽',
 '정원속화실담다',
 '트렌드바이미플라워',
 '29 x 냥초',
 '___알렉스',
 '빨간머리 앤',
 "아이홉 맥주공방 (I'HOP)",
 '이도경',
 '방쿤',
 '정원속화실담다',
 '제이몬티 베이킹랩',
 '비비우드스튜디오',
 'SO뮤직',
 '이지훈',
 '한국술집 안씨막걸리',
 'HappyBaker',
 '강남클라이밍',
 '남대문실탄사격장',
 '파파쿡',
 '이타(ITA)',
 '오하나_메이',
 '제이몬티 베이킹랩',
 '공유카페',
 '추월',
 '공유카페',
 '트렌드바이미플라워',
 '예빈당',
 '보니보니베이킹스튜디오',
 '본따블르',
 'Art Space',
 '권영세클라이밍센터',
 '블러썸앤어나더',
 'mok',
 'OvenPal',
 '디세',
 'OvenPal',
 '솔메',
 '이든',
 '쩡쌤',
 '현우',
 '브라보서프',
 '공유카페',
 '여우학교',
 '롤앤쿠키',
 '도란도자기공방',
 '스무:디 (20diy)',
 '샤뜰리에화실',
 '조수정폴댄스',
 'Ray Kim',
 '아브릴 베이킹 스튜디오',
 '레이나스튜디오',
 'POTTRING',
 '청년건물주 제이든',
 '프롬디센트',
 '다미',
 '벨로르플라워',
 '와플목공방',
 '헤펠레목공방',
 '피피아트',
 '당근자판기',
 '타이거 포터리',
 'WM트릭킹짐',
 'dojagihaeum',
 '담따프레시',
 '카이카스튜디오',
 '재테크 神',
 'simcook',
 '리브모아',
 '스튜디오 글로위',
 '보니보니베이킹스튜디오',
 'NPC 지니',
 'flake',
 '보니보니베이킹스튜디오',
 'Anna',
 'Stan',
 '호비뮤직',
 '드로잉클럽',
 '방쿤',
 'Tenuto',
 '전상면',
 '유일향',
 '두부',
 '아브릴 베이킹 스튜디오',
 '카이카스튜디오',
 '마크컬렉션'